# Create annual and temperature based predictions from the best models

In this notebook we'll create predictions for surface type specific annual budget and temperature specific flux using the full model equation and by setting $x_{i,j} = 1$ for each surface type j yielding a out-of-model equation

$$
E(\ln{F_i} | x_{i,j}=1) = (\alpha + \gamma_j) + (\beta + \delta_j) \frac{T_{\mathrm{air,i,j}} - 10^{\circ} \mathrm{C}}{10^{\circ} \mathrm{C}}
$$

In [1]:
import cloudpickle
import pymc as pm
import arviz as az
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

%matplotlib widget
%load_ext autoreload
%autoreload 2

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# Load data to extract soil class names

In [2]:
data_n2o = pd.read_csv('data/inference_data_n2o.csv', index_col='time')
data_n2o.index = pd.to_datetime(data_n2o.index)

data_ch4 = pd.read_csv('data/inference_data_ch4.csv', index_col='time')
data_ch4.index = pd.to_datetime(data_ch4.index)

# Open the best models

In [3]:
with open('models/full_model_n2o_st9_mutable.pkl','rb') as buff:
    model_n2o = cloudpickle.load(buff)

with open('models/full_model_ch4_st6_mutable.pkl','rb') as buff:
    model_ch4 = cloudpickle.load(buff)

In [4]:
new_st_9 = np.array(['Dead wood','Harvest residue','Exposed peat','Litter','Bottom layer (mosses)','Field layer','Ditch (water surface)', 'Living tree', 'Plant covered ditch'])
new_st_6 = np.array(['Exposed peat', 'Ditch (water surface)', 'Plant covered ditch', 'Dead wood and residue', 'Litter', 'Field layer and trees',])

# Load T<sub>air</sub> data for annual budget

In [5]:
Tair = pd.read_csv('data/T_air_gapfilled.csv')
T = Tair['T_air'].values
T = T.reshape(T.shape[0], 1)
T_air_annual_st_9 = np.repeat(T, repeats=9, axis=1)
T_air_annual_st_6 = np.repeat(T, repeats=6, axis=1)

# Define temperature vector for T specific surface type flux calculation

In [6]:
T_response_st_9 = np.repeat(np.linspace(np.min(T), np.max(T), 101).reshape(101,1), repeats=9, axis=1)

T_response_st_6 = np.repeat(np.linspace(np.min(T), np.max(T), 101).reshape(101,1), repeats=6, axis=1)

# Define surface type fractions

In [7]:
# # copy paste from GHG_site_level_annual_flux

# X_st_9 = np.array([0.22831063575874028,
#  0.07912675405730267,
#  0.29005355713693703,
#  0.19903329159584093,
#  0.014206532335570385,
#  0.11784682410999092,
#  0.018604968072915225,
#  0.04190943651265434,
#  0.010801791670784143,
#  0.00010620874926418247])

# X_st_9 = X_st_9.reshape(1, -1)
# #X_st_9 = X_st_9[:, :-1]

In [8]:
model_n2o['model_res']['posterior'].st

<xarray.DataArray 'st' (st: 9)>
array(['Fallen tree', 'Dry branch', 'Exposed peat', 'Light moss', 'Dark moss',
       'Green vegetation', 'Ditch', 'Tree', 'Covered ditch'], dtype='<U16')
Coordinates:
  * st       (st) <U16 'Fallen tree' 'Dry branch' ... 'Tree' 'Covered ditch'

In [9]:
# Edit the copy pasted values (the last one is the instrument class which we don't want)

X_st_9 = np.array([0.22831063575874028,
 0.07912675405730267,
 0.29005355713693703,
 0.19903329159584093,
 0.014206532335570385,
 0.11784682410999092,
 0.018604968072915225,
 0.04190943651265434,
 0.010801791670784143,
])

X_st_9 = X_st_9.reshape(1, -1)

In [10]:
X_st_9

array([[0.22831064, 0.07912675, 0.29005356, 0.19903329, 0.01420653,
        0.11784682, 0.01860497, 0.04190944, 0.01080179]])

In [11]:
model_ch4['model_res']['posterior'].st

<xarray.DataArray 'st' (st: 6)>
array(['Exposed peat', 'Ditch', 'Covered ditch', 'Logging residue',
       'Light moss', 'Green vegetation and trees'], dtype='<U26')
Coordinates:
  * st       (st) <U26 'Exposed peat' 'Ditch' ... 'Green vegetation and trees'

In [12]:
X_st_6 = np.array([0.29005355713693703,
                   0.018604968072915225,
                   0.010801791670784143,
                   0.22831063575874028 + 0.07912675405730267,
                   0.19903329159584093,
                   0.11784682410999092 + 0.04190943651265434
                   ])
X_st_6 = X_st_6.reshape(1, -1)

# N<sub>2</sub>O predictions

In [13]:
ind = ((data_n2o.T_air <= 11) & (data_n2o.T_air >= 9))
n2o_tenC_mean, n2o_tenC_std = ((data_n2o.loc[ind, 'F_N2O_ln']).mean(), (data_n2o.loc[ind, 'F_N2O_ln']).std())

## Annual T<sub>air</sub> for the whole plot

In [14]:
gamma_std = 2.0
lambda_beta = 1.0
lambda_delta = 1.0
lambda_epsilon = 1.0
alpha_mu_mean, alpha_mu_std = n2o_tenC_mean, n2o_tenC_std

coords = {'st': new_st_9, 'T_id': np.arange(T_air_annual_st_9.shape[0])}
with pm.Model(coords=coords) as new_model:
    T_air = T_air_annual_st_9
    st_frac = np.repeat(X_st_9, repeats=T_air_annual_st_9.shape[0], axis=0)
    alpha = pm.Normal("alpha", mu=alpha_mu_mean, sigma=alpha_mu_std)
    beta = pm.Exponential('beta', lambda_beta)

    gamma_mu = pm.Normal("gamma_mu", mu=0, sigma=1)
    gamma = pm.Normal("gamma", mu=gamma_mu, sigma=gamma_std, dims='st') 

    delta = pm.Exponential("delta", lambda_delta, dims='st')

    epsilon = pm.Exponential("sigma", lambda_epsilon)

    pred = pm.Normal("pred", mu = alpha + beta*(T_air[:, 0]-10)/10 + pm.math.sum(gamma*st_frac + delta*(T_air-10)/10*st_frac, axis=1), sigma=epsilon)

    annual_n2o = pm.sample_posterior_predictive(model_n2o['model_res'].posterior, var_names=["pred"], predictions=True)

Sampling: [pred]


In [15]:
annual_n2o.predictions['sigma'] = (["chain", "draw"], model_n2o['model_res'].posterior.sigma.data)

In [16]:
annual_n2o.predictions['pred_real_bias_corrected'] = (["chain", "draw", "pred_dim_2"], (np.exp(annual_n2o.predictions.pred.data + annual_n2o.predictions.sigma.data[:, :, None]**2.0/2.0)))

### Rename dims

In [17]:
dates = pd.date_range(datetime(2022,5,1), datetime(2022,11,16), freq="30min")

In [18]:
annual_n2o = annual_n2o.predictions
annual_n2o = annual_n2o.rename_dims({'pred_dim_2': 'time'})
annual_n2o = annual_n2o.rename_vars({'pred_dim_2': 'time'})
annual_n2o['time'] = dates

### Save

In [19]:
annual_n2o.to_netcdf('data/n2o_pred_annual_T_air.nc')

## T response for each surface type

In [20]:
gamma_std = 2.0
lambda_beta = 1.0
lambda_delta = 1.0
lambda_epsilon = 1.0
alpha_mu_mean, alpha_mu_std = n2o_tenC_mean, n2o_tenC_std

coords = {'st': new_st_9, 'T_id': np.arange(T_response_st_9.shape[0])}
with pm.Model(coords=coords) as new_model:
    T_air = T_response_st_9
    alpha = pm.Normal("alpha", mu=alpha_mu_mean, sigma=alpha_mu_std)
    beta = pm.Exponential('beta', lambda_beta)

    gamma_mu = pm.Normal("gamma_mu", mu=0, sigma=1)
    gamma = pm.Normal("gamma", mu=gamma_mu, sigma=gamma_std, dims='st') 

    delta = pm.Exponential("delta", lambda_delta, dims='st')

    epsilon = pm.Exponential("sigma", lambda_epsilon)

    pred = pm.Normal("pred", mu = (alpha+gamma) + (beta+delta)*(T_air-10)/10, sigma=epsilon)

    T_response_n2o = pm.sample_posterior_predictive(model_n2o['model_res'], var_names=["pred"], predictions=True)

Sampling: [pred]


In [21]:
T_response_n2o

Inference data with groups:
	> predictions

### Add sigma and bias corrected prediction

In [22]:
T_response_n2o.predictions['sigma'] = (["chain", "draw"], model_n2o['model_res'].posterior.sigma.data)

In [23]:
T_response_n2o.predictions['pred_real_bias_corrected'] = (["chain", "draw", "pred_dim_2", "pred_dim_3"], (np.exp(T_response_n2o.predictions.pred.data + T_response_n2o.predictions.sigma.data[:, :, None, None]**2.0/2.0)))

### Rename dims

In [24]:
T_response_n2o = T_response_n2o.predictions
T_response_n2o = T_response_n2o.rename_dims({'pred_dim_2': 'Tair'})
T_response_n2o = T_response_n2o.rename_vars({'pred_dim_2': 'Tair'})
T_response_n2o['Tair'] = T_response_st_9[:,0].flatten()

In [25]:
T_response_n2o = T_response_n2o.rename_dims({'pred_dim_3': 'soil_classes'})
T_response_n2o = T_response_n2o.rename_vars({'pred_dim_3': 'soil_classes'})
T_response_n2o['soil_classes'] = new_st_9

### Save

In [26]:
T_response_n2o.to_netcdf('data/n2o_pred_T_response.nc')

## Annual T for each surface type

In [27]:
gamma_std = 2.0
lambda_beta = 1.0
lambda_delta = 1.0
lambda_epsilon = 1.0
alpha_mu_mean, alpha_mu_std = n2o_tenC_mean, n2o_tenC_std

coords = {'st': new_st_9, 'T_id': np.arange(T_air_annual_st_9.shape[0])}
with pm.Model(coords=coords) as new_model:
    T_air = T_air_annual_st_9
    alpha = pm.Normal("alpha", mu=alpha_mu_mean, sigma=alpha_mu_std)
    beta = pm.Exponential('beta', lambda_beta)

    gamma_mu = pm.Normal("gamma_mu", mu=0, sigma=1)
    gamma = pm.Normal("gamma", mu=gamma_mu, sigma=gamma_std, dims='st') 

    delta = pm.Exponential("delta", lambda_delta, dims='st')

    epsilon = pm.Exponential("sigma", lambda_epsilon)

    pred = pm.Normal("pred", mu = (alpha+gamma) + (beta+delta)*(T_air-10)/10, sigma=epsilon)

    annual_st_n2o = pm.sample_posterior_predictive(model_n2o['model_res'], var_names=["pred"], predictions=True)

Sampling: [pred]


### Add sigma and bias corrected prediction

In [28]:
annual_st_n2o.predictions['sigma'] = (["chain", "draw"], model_n2o['model_res'].posterior.sigma.data)

In [29]:
annual_st_n2o.predictions['pred_real_bias_corrected'] = (["chain", "draw", "pred_dim_2", "pred_dim_3"], (np.exp(annual_st_n2o.predictions.pred.data + annual_st_n2o.predictions.sigma.data[:, :, None, None]**2.0/2.0)))

### Rename dims

In [30]:
annual_st_n2o = annual_st_n2o.predictions
annual_st_n2o = annual_st_n2o.rename_dims({'pred_dim_2': 'time'})
annual_st_n2o = annual_st_n2o.rename_vars({'pred_dim_2': 'time'})
annual_st_n2o['time'] = dates

In [31]:
annual_st_n2o = annual_st_n2o.rename_dims({'pred_dim_3': 'soil_classes'})
annual_st_n2o = annual_st_n2o.rename_vars({'pred_dim_3': 'soil_classes'})
annual_st_n2o['soil_classes'] = new_st_9

In [32]:
annual_st_n2o.to_netcdf('data/annual_st_n2o.nc')

# CH<sub>4</sub> predictions

In [33]:
ind = ((data_ch4.T_air <= 11) & (data_ch4.T_air >= 9))
ch4_tenC_mean, ch4_tenC_std = ((data_ch4.loc[ind, 'F_CH4_ln']).mean(), (data_ch4.loc[ind, 'F_CH4_ln']).std())

## Annual T<sub>air</sub>

In [34]:
gamma_std = 2.0
lambda_beta = 1.0
lambda_delta = 1.0
lambda_epsilon = 1.0
alpha_mu_mean, alpha_mu_std = ch4_tenC_mean, ch4_tenC_std

coords = {'st': new_st_6, 'T_id': np.arange(T_air_annual_st_6.shape[0])}
with pm.Model(coords=coords) as new_model:
    T_air = T_air_annual_st_6
    st_frac = np.repeat(X_st_6, repeats=T_air_annual_st_6.shape[0], axis=0)
    alpha = pm.Normal("alpha", mu=alpha_mu_mean, sigma=alpha_mu_std)
    beta = pm.Exponential('beta', lambda_beta)

    gamma_mu = pm.Normal("gamma_mu", mu=0, sigma=1)
    gamma = pm.Normal("gamma", mu=gamma_mu, sigma=gamma_std, dims='st') 

    delta = pm.Exponential("delta", lambda_delta, dims='st')

    epsilon = pm.Exponential("sigma", lambda_epsilon)

    pred = pm.Normal("pred", mu = alpha + beta*(T_air[:, 0]-10)/10 + pm.math.sum(gamma*st_frac + delta*(T_air-10)/10*st_frac, axis=1), sigma=epsilon)

    annual_ch4 = pm.sample_posterior_predictive(model_ch4['model_res'], var_names=["pred"], predictions=True)

Sampling: [pred]


### Add sigma and bias corrected prediction

In [35]:
annual_ch4.predictions['sigma'] = (["chain", "draw"], model_ch4['model_res'].posterior.sigma.data)

In [36]:
annual_ch4.predictions['pred_real_bias_corrected'] = (["chain", "draw", "pred_dim_2"], (np.exp(annual_ch4.predictions.pred.data + annual_ch4.predictions.sigma.data[:, :, None]**2.0/2.0)-10))

### Rename dims

In [37]:
annual_ch4 = annual_ch4.predictions
annual_ch4 = annual_ch4.rename_dims({'pred_dim_2': 'time'})
annual_ch4 = annual_ch4.rename_vars({'pred_dim_2': 'time'})
annual_ch4['time'] = dates

### Save

In [38]:
annual_ch4.to_netcdf('data/ch4_pred_annual_T_air.nc')

In [39]:
gamma_std = 2.0
lambda_beta = 1.0
lambda_delta = 1.0
lambda_epsilon = 1.0
alpha_mu_mean, alpha_mu_std = ch4_tenC_mean, ch4_tenC_std

coords = {'st': new_st_6, 'T_id': np.arange(T_response_st_6.shape[0])}
with pm.Model(coords=coords) as new_model:
    T_air = T_response_st_6
    alpha = pm.Normal("alpha", mu=alpha_mu_mean, sigma=alpha_mu_std)
    beta = pm.Exponential('beta', lambda_beta)

    gamma_mu = pm.Normal("gamma_mu", mu=0, sigma=1)
    gamma = pm.Normal("gamma", mu=gamma_mu, sigma=gamma_std, dims='st') 

    delta = pm.Exponential("delta", lambda_delta, dims='st')

    epsilon = pm.Exponential("sigma", lambda_epsilon)

    pred = pm.Normal("pred", mu = (alpha+gamma) + (beta+delta)*(T_air-10)/10, sigma=epsilon)

    T_response_ch4 = pm.sample_posterior_predictive(model_ch4['model_res'], var_names=["pred"], predictions=True)

Sampling: [pred]


### Add sigma and bias corrected prediction

In [40]:
T_response_ch4.predictions['sigma'] = (["chain", "draw"], model_ch4['model_res'].posterior.sigma.data)

In [41]:
T_response_ch4.predictions['pred_real_bias_corrected'] = (["chain", "draw", "pred_dim_2", "pred_dim_3"], (np.exp(T_response_ch4.predictions.pred.data + T_response_ch4.predictions.sigma.data[:, :, None, None]**2.0/2.0)-10))

### Rename dims

In [42]:
T_response_ch4 = T_response_ch4.predictions
T_response_ch4 = T_response_ch4.rename_dims({'pred_dim_2': 'Tair'})
T_response_ch4 = T_response_ch4.rename_vars({'pred_dim_2': 'Tair'})
T_response_ch4['Tair'] = T_response_st_6[:,0].flatten()

In [43]:
T_response_ch4 = T_response_ch4.rename_dims({'pred_dim_3': 'soil_classes'})
T_response_ch4 = T_response_ch4.rename_vars({'pred_dim_3': 'soil_classes'})
T_response_ch4['soil_classes'] = new_st_6

### Save

In [44]:
T_response_ch4.to_netcdf('data/ch4_pred_T_response.nc')

## Annual T for each surface type

In [45]:
gamma_std = 2.0
lambda_beta = 1.0
lambda_delta = 1.0
lambda_epsilon = 1.0
alpha_mu_mean, alpha_mu_std = ch4_tenC_mean, ch4_tenC_std

coords = {'st': new_st_6, 'T_id': np.arange(T_air_annual_st_6.shape[0])}
with pm.Model(coords=coords) as new_model:
    T_air = T_air_annual_st_6
    alpha = pm.Normal("alpha", mu=alpha_mu_mean, sigma=alpha_mu_std)
    beta = pm.Exponential('beta', lambda_beta)

    gamma_mu = pm.Normal("gamma_mu", mu=0, sigma=1)
    gamma = pm.Normal("gamma", mu=gamma_mu, sigma=gamma_std, dims='st') 

    delta = pm.Exponential("delta", lambda_delta, dims='st')

    epsilon = pm.Exponential("sigma", lambda_epsilon)

    pred = pm.Normal("pred", mu = (alpha+gamma) + (beta+delta)*(T_air-10)/10, sigma=epsilon)

    annual_st_ch4 = pm.sample_posterior_predictive(model_ch4['model_res'], var_names=["pred"], predictions=True)

Sampling: [pred]


### Add sigma and bias corrected prediction

In [46]:
annual_st_ch4.predictions['sigma'] = (["chain", "draw"], model_ch4['model_res'].posterior.sigma.data)

In [47]:
annual_st_ch4.predictions['pred_real_bias_corrected'] = (["chain", "draw", "pred_dim_2", "pred_dim_3"], (np.exp(annual_st_ch4.predictions.pred.data + annual_st_ch4.predictions.sigma.data[:, :, None, None]**2.0/2.0)-10))

### Rename dims

In [48]:
annual_st_ch4 = annual_st_ch4.predictions
annual_st_ch4 = annual_st_ch4.rename_dims({'pred_dim_2': 'time'})
annual_st_ch4 = annual_st_ch4.rename_vars({'pred_dim_2': 'time'})
annual_st_ch4['time'] = dates

In [49]:
annual_st_ch4 = annual_st_ch4.rename_dims({'pred_dim_3': 'soil_classes'})
annual_st_ch4 = annual_st_ch4.rename_vars({'pred_dim_3': 'soil_classes'})
annual_st_ch4['soil_classes'] = new_st_6

In [50]:
annual_st_ch4.to_netcdf('data/annual_st_ch4.nc')